In [2]:
import sys
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
import numpy as np
from backend import Washer  # 后端处理

import cgitb
cgitb.enable(format='text')  # 显示多线程错误信息

Using TensorFlow backend.


Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


In [3]:
# 全局变量
PAUSE = 0

# 将图像处理工作放在一个QObject子类中完成，用于QThread多线程实现
class Worker(QObject):

    call_todo = pyqtSignal(str)  # 用于触发worker的后台运行
    trigger = pyqtSignal()  # 每处理完一帧触发显示
    clear = pyqtSignal()    # 触发清空输入缓存的信号

    def __init__(self):
        super().__init__()
        self.washing_time = 0
        self.frame_to_show = np.zeros((1024, 576, 3), dtype=np.uint8)

    @pyqtSlot(str)  # 使signal-slot的绑定关系与moveToThread的先后顺序无关
    def start(self, video_name):
        print('subthread ID: ', int(QThread.currentThreadId()))  # 打印子线程号
        lil_washer = Washer(video_path=video_name)
        self.frame_to_show = np.zeros((1024, 576, 3), dtype=np.uint8)
        self.washing_time = 0  # 洗手总计时间(1/24×帧数，单位为s)
        self.video_end = 0
        # VIDEO_PATH = 'C:/Users/Minghao/Desktop/gopro1/TEST1.MP4'
        while 1:
            # 07/24/2019: 将状态机的flag信号添加到输出列表中:self.machine_state，目的是确保flag=0的时候才能确认付款
            if PAUSE == 1:
                pass
            elif PAUSE == -1:
                lil_washer.video_cap.release()
                break
            elif PAUSE == 0:
                self.frame_to_show, self.washing_time, self.video_end = lil_washer.predict()
                self.trigger.emit()
                if self.video_end:  # 视频播放到了最后一帧
                    self.video_end = 0
                    self.clear.emit()

In [4]:
class FrontEnd(QWidget):

    def __init__(self):
        super().__init__()
        self.video_name = 'C:/Users/Minghao/Desktop/gopro1/TEST1.MP4'
        self.initUI()

    def initUI(self):
        # 为图像处理建立后台线程
        self.workThread = QThread()
        self.workThread.start()
        self.worker = Worker()  # 例化一个worker对象
        self.worker.moveToThread(self.workThread)
        self.worker.trigger.connect(self.showFrame)  # 每处理完一帧，打印
        self.worker.clear.connect(self.restart)
        self.worker.call_todo.connect(self.worker.start)

        # 以下是控件及其功能设计
        # 获取显示器分辨率
        self.desktop = QApplication.desktop()
        self.screenRect = self.desktop.screenGeometry()
        self.width = self.screenRect.width()  # 屏幕宽度
        self.height = self.screenRect.height()  # 屏幕高度

        # self.setGeometry(int(self.width/4), int(self.height/4), int(self.width/4*9/16), int(self.height/4*16/9))  # 窗口初始大小为屏幕的1/4
        # self.setMinimumSize(1280, 720)
#         self.setGeometry(300, 300, 1200, 700)
        self.setWindowTitle("洗手检测demo")

        self.label_show_image = QLabel()  # 显示摄像头画面的控件，DragLabel是继承自QLabel的自定义类，实现了鼠标选区功能
        self.label_show_image.setFixedSize(1024, 576)

        self.loadVideoButton = QPushButton("视频输入")
        # self.loadVideoButton.setText(u"选择视频")
        self.loadVideoButton.clicked.connect(self.loadVideo)
        self.loadVideoButton.setFixedSize(400, 50)

        self.startButton = QPushButton("开始检测")
        self.startButton.clicked.connect(self.work)
        self.startButton.setCheckable(True)  # 设为可以按下和弹起的形式
        self.startButton.setFixedSize(400, 50)

        # 进度条和刷新进度条用的计时器
        self.pbar = QProgressBar()
        self.pbar.setRange(0, 10)
        self.pbar.setFixedSize(900, 50)
        self.timer = QBasicTimer()
        self.timer.start(100, self)


        # 控件布局设计：QGridLayout
        main_layout = QGridLayout(self)  # 整体使用网格布局
        main_layout.addWidget(self.label_show_image, 0, 0, 9, 16)
        main_layout.addWidget(self.loadVideoButton, 9, 0, 2, 8)
        main_layout.addWidget(self.startButton, 9, 9, 2, 8)
        main_layout.addWidget(self.pbar, 11, 0, 2, 16)
        main_layout.setAlignment(Qt.AlignCenter)
        self.setLayout(main_layout)

        self.showFrame()

        # 设置整个窗口的背景颜色为白色而不是默认的灰色
        bgColor = QPalette()
        bgColor.setColor(self.backgroundRole(), QColor(255, 255, 255))
        self.setPalette(bgColor)

    # 重写QObject的timerEvent方法，用于刷新显示进度条
    def timerEvent(self, event):
        if self.worker.washing_time >= 10:
            self.time = 10
        else:
            self.time = self.worker.washing_time
        self.pbar.setValue(self.time)
        self.pbar.update()


    def loadVideo(self):
         video_name = QFileDialog.getOpenFileName(self, caption="open file dialog", directory="C:/Users/Minghao/Desktop/gopro")
         self.video_name = video_name[0]

    def keyPressEvent(self, keyEvent):
        global PAUSE
        if keyEvent.key() == Qt.Key_Escape:
            print(keyEvent.key())
            self.close()
        elif keyEvent.key() == Qt.Key_P:  # 按下P键暂停/继续
            PAUSE = PAUSE ^ 1
        elif keyEvent.key() == Qt.Key_Q:  # 按下Q键停止当前视频
            PAUSE = -1

    # 第一个按钮：开始结账/确认付款
    def work(self):
        global PAUSE
        PAUSE = 0
        self.worker.call_todo.emit(self.video_name)


    def showFrame(self):
        self.label_show_image.setScaledContents(True)
        IMG = self.worker.frame_to_show
        frame_height, frame_width, channel = IMG.shape
        bytesPerline = 3 * frame_width
        img = QImage(IMG.data, frame_width, frame_height, bytesPerline,
                     QImage.Format_RGB888).rgbSwapped()
        # 根据宽高比设置待显示图像的尺寸（使用QImage的scaled方法）
        widget_width = self.label_show_image.width()
        widget_height = self.label_show_image.height()
        frame_ratio = float(frame_width) / float(frame_height)
        widget_ratio = float(widget_width) / float(widget_height)
        if frame_ratio <= widget_ratio:
            adjusted_height = widget_height
            adjusted_width = int(widget_height * frame_ratio)
        else:  # frame_ratio >= widget_ratio
            adjusted_width = widget_width
            adjusted_height = int(widget_width / frame_ratio)
        img = QPixmap.fromImage(img).scaled(adjusted_width, adjusted_height)
        self.label_show_image.setPixmap(img)



    def restart(self):  # 当前视频播放完毕，重播一遍
        # 如果本次扫码失败,弹出对话框进行提示
        self.worker.call_todo.emit(self.video_name)

In [5]:
if __name__ == "__main__":
    print('parent thread ID: ', int(QThread.currentThreadId()))  # 打印主线程ID
    app = QApplication(sys.argv)
    example = FrontEnd()
    example.show()
    sys.exit(app.exec_())

parent thread ID:  28192
subthread ID:  860
frame ID:  1
frame ID:  2
frame ID:  3
frame ID:  4
frame ID:  5
frame ID:  6
frame ID:  7
frame ID:  8
frame ID:  9
frame ID:  10
frame ID:  11
frame ID:  12
frame ID:  19
frame ID:  20
frame ID:  21
frame ID:  22
frame ID:  23
frame ID:  24
frame ID:  25
frame ID:  26
frame ID:  27
frame ID:  28
frame ID:  29
frame ID:  30
frame ID:  31
frame ID:  32
frame ID:  33
frame ID:  34
frame ID:  35
frame ID:  36
frame ID:  37
frame ID:  38
frame ID:  39
frame ID:  40
frame ID:  41
frame ID:  42
frame ID:  43
frame ID:  44
frame ID:  45
frame ID:  46
frame ID:  47
frame ID:  48
frame ID:  49
frame ID:  50
frame ID:  51
frame ID:  52
frame ID:  53
frame ID:  54
frame ID:  55
frame ID:  56
frame ID:  57
frame ID:  58
frame ID:  59
frame ID:  60
frame ID:  61
frame ID:  62
frame ID:  63
frame ID:  64
frame ID:  65
frame ID:  66
frame ID:  67
frame ID:  68
frame ID:  69
frame ID:  70
frame ID:  71
frame ID:  72
frame ID:  73
frame ID:  74
frame ID:  75

washing time:  5.750000000000006
frame ID:  259
washing time:  5.791666666666673
frame ID:  260
washing time:  5.83333333333334
frame ID:  261
washing time:  5.875000000000007
frame ID:  262
washing time:  5.916666666666674
frame ID:  263
washing time:  5.958333333333341
frame ID:  264
washing time:  6.000000000000008
frame ID:  265
washing time:  6.041666666666675
frame ID:  266
washing time:  6.083333333333342
frame ID:  267
washing time:  6.125000000000009
frame ID:  268
washing time:  6.166666666666676
frame ID:  269
washing time:  6.208333333333343
frame ID:  270
washing time:  6.25000000000001
frame ID:  271
washing time:  6.291666666666677
frame ID:  272
washing time:  6.333333333333344
frame ID:  273
washing time:  6.375000000000011
frame ID:  274
washing time:  6.416666666666678
frame ID:  275
washing time:  6.458333333333345
frame ID:  276
washing time:  6.5000000000000115
frame ID:  277
washing time:  6.5416666666666785
frame ID:  278
washing time:  6.5833333333333455
frame 

SystemExit: 0

c:\users\minghao\anaconda3\envs\tf36\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


washing time:  9.999999999999993
frame ID:  361
washing time:  10.041666666666659
frame ID:  362
washing time:  10.083333333333325
frame ID:  363
washing time:  10.124999999999991
frame ID:  364
washing time:  10.166666666666657
frame ID:  365
washing time:  10.208333333333323
frame ID:  366
washing time:  10.24999999999999
frame ID:  367
washing time:  10.291666666666655
frame ID:  368
washing time:  10.333333333333321
frame ID:  369
washing time:  10.374999999999988
frame ID:  370
washing time:  10.416666666666654
frame ID:  371
washing time:  10.45833333333332
frame ID:  372
washing time:  10.499999999999986
frame ID:  373
washing time:  10.541666666666652
frame ID:  374
washing time:  10.583333333333318
frame ID:  375
washing time:  10.624999999999984
frame ID:  376
washing time:  10.66666666666665
frame ID:  377
washing time:  10.708333333333316
frame ID:  378
washing time:  10.749999999999982
frame ID:  379
washing time:  10.791666666666648
frame ID:  380
washing time:  10.833333

washing time:  15.916666666666575
frame ID:  591
washing time:  15.958333333333242
frame ID:  592
washing time:  15.999999999999908
frame ID:  593
washing time:  16.041666666666575
frame ID:  594
washing time:  16.083333333333243
frame ID:  595
washing time:  16.12499999999991
frame ID:  596
washing time:  16.16666666666658
frame ID:  597
washing time:  16.208333333333247
frame ID:  598
washing time:  16.249999999999915
frame ID:  599
washing time:  16.291666666666583
frame ID:  600
washing time:  16.33333333333325
frame ID:  601
washing time:  16.37499999999992
frame ID:  602
washing time:  16.416666666666586
frame ID:  603
washing time:  16.458333333333254
frame ID:  604
washing time:  16.499999999999922
frame ID:  605
washing time:  16.54166666666659
frame ID:  606
washing time:  16.583333333333258
frame ID:  607
frame ID:  608
frame ID:  609
frame ID:  610
frame ID:  611
frame ID:  612
frame ID:  613
frame ID:  614
frame ID:  615
frame ID:  616
frame ID:  617
frame ID:  618
frame I

washing time:  21.666666666666735
frame ID:  834
washing time:  21.708333333333403
frame ID:  835
washing time:  21.75000000000007
frame ID:  836
washing time:  21.79166666666674
frame ID:  837
washing time:  21.833333333333407
frame ID:  838
washing time:  21.875000000000075
frame ID:  839
washing time:  21.916666666666742
frame ID:  840
washing time:  21.95833333333341
frame ID:  841
washing time:  22.000000000000078
frame ID:  842
washing time:  22.041666666666746
frame ID:  843
washing time:  22.083333333333414
frame ID:  844
washing time:  22.12500000000008
frame ID:  845
washing time:  22.16666666666675
frame ID:  846
washing time:  22.208333333333417
frame ID:  847
washing time:  22.250000000000085
frame ID:  848
washing time:  22.291666666666753
frame ID:  849
washing time:  22.33333333333342
frame ID:  850
washing time:  22.37500000000009
frame ID:  851
washing time:  22.416666666666757
frame ID:  852
washing time:  22.458333333333425
frame ID:  853
washing time:  22.500000000

washing time:  28.75000000000027
frame ID:  1004
washing time:  28.791666666666938
frame ID:  1005
washing time:  28.833333333333606
frame ID:  1006
washing time:  28.875000000000274
frame ID:  1007
washing time:  28.91666666666694
frame ID:  1008
washing time:  28.95833333333361
frame ID:  1009
washing time:  29.000000000000277
frame ID:  1010
washing time:  29.041666666666945
frame ID:  1011
washing time:  29.083333333333613
frame ID:  1012
washing time:  29.12500000000028
frame ID:  1013
washing time:  29.16666666666695
frame ID:  1014
washing time:  29.208333333333616
frame ID:  1015
washing time:  29.250000000000284
frame ID:  1016
washing time:  29.291666666666952
frame ID:  1017
washing time:  29.33333333333362
frame ID:  1018
washing time:  29.375000000000288
frame ID:  1019
washing time:  29.416666666666956
frame ID:  1020
washing time:  29.458333333333623
frame ID:  1021
washing time:  29.50000000000029
frame ID:  1022
washing time:  29.54166666666696
frame ID:  1023
washing 